<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-13 11:21:38
-------------------
qualified stocks: 93
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  2.51 L
Current:  1.33 C
-------------------
Today PnL: 64.55 K (0.49%)
Current PnL: -21.77 L (-14.81%)
CY Booked + Current PnL: -13.50 L (-9.18%)
-------------------
Total profit:  2.61 L
Total loss:  -24.38 L
-------------------
Total Booked + Current PnL: 13.48 L (11.23%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.21%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 81.25 L (61.04%)
Deployed:  1.20 C
Current:  1.33 C
CAGR/XIRR %: 6.64%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,756.01,-18.25,H-LC,6.04,112288.0,7976.0,8669.0,5.21,7.65,7.72,15.96,11.0,0.92,0.86,25.84,X5K,ATH,METALS
68,SBIN,760.30,863.00,-11.34,M-LC,3.86,218400.0,16920.0,10287.0,0.43,8.40,4.71,13.51,61.0,1.64,1.67,22.05,XY25,NTT,BANKS
4,APOLLOHOSP,7098.95,8285.00,-15.30,H-LC,5.59,169169.0,12992.0,13094.0,6.26,8.32,7.74,16.71,19.0,0.99,1.30,27.04,X40N,BTT,HEALTHCARE
55,MEDANTA,1087.93,1486.00,10.86,H-MC,13.94,158189.0,31989.0,14190.0,-1.20,25.35,8.97,36.59,111.0,2.25,1.21,41.04,XY24,NTT,HEALTHCARE
51,LALPATHLAB,2838.53,3545.00,-25.14,H-SC,9.63,223584.0,30564.0,17484.0,0.98,15.83,7.82,24.89,134.0,1.75,1.71,42.88,X40N,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,DABUR,505.20,735.00,-10.32,H-MC,3.74,195195.0,-1833.0,91449.0,-0.48,-0.93,46.85,45.49,103.0,-0.02,1.49,11.99,XY24,BTT,FMCG
56,NATIONALUM,189.63,247.44,-49.46,H-MC,2.03,101495.0,991.0,29647.0,2.67,0.99,29.21,30.49,80.0,0.03,0.78,33.71,MH,ATH,METALS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-20.81,H-LC,2.26,211460.0,6960.0,34447.0,1.44,3.40,16.29,20.25,10.0,0.20,1.62,12.43,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-13.19,H-LC,2.82,169225.0,-32161.0,119777.0,0.42,-15.97,70.78,43.51,27.0,-0.27,1.30,17.80,X40,BTT,PAINTS
57,NESTLEIND,2268.60,2755.0,-17.26,H-LC,3.59,127401.0,-138025.0,194936.0,-0.20,-52.00,153.01,21.44,8.0,-0.71,0.98,2.81,XY25,NTT,FMCG
53,LTIM,5564.16,7230.2,-7.80,H-LC,3.80,182718.0,-17592.0,77564.0,-0.62,-8.78,42.45,29.94,17.0,-0.23,1.40,26.23,X200,ATH,IT
4,APOLLOHOSP,7098.95,8285.0,-15.30,H-LC,5.59,169169.0,12992.0,13094.0,6.26,8.32,7.74,16.71,19.0,0.99,1.30,27.04,X40N,BTT,HEALTHCARE


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,APOLLOHOSP,7098.95,8285.00,-15.30,H-LC,5.59,169169.0,12992.0,13094.0,6.26,8.32,7.74,16.71,19.0,0.99,1.30,27.04,X40N,BTT,HEALTHCARE
36,HINDALCO,651.95,756.01,-18.25,H-LC,6.04,112288.0,7976.0,8669.0,5.21,7.65,7.72,15.96,11.0,0.92,0.86,25.84,X5K,ATH,METALS
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,21.85,72192.0,-41357.0,81411.0,5.17,-36.42,112.77,35.27,268.0,-0.51,0.55,76.69,XR,NTT,HOTELS
35,HEROMOTOCO,4311.81,5949.07,-9.65,H-MC,7.43,167685.0,16772.0,40529.0,3.14,11.11,24.17,37.97,92.0,0.41,1.28,39.23,AR,ATH,AUTO
56,NATIONALUM,189.63,247.44,-49.46,H-MC,2.03,101495.0,991.0,29647.0,2.67,0.99,29.21,30.49,80.0,0.03,0.78,33.71,MH,ATH,METALS


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BATAINDIA,1550.24,2096.0,-38.58,M-SC,6.55,91881.0,-36789.0,82086.0,-2.54,-28.59,89.34,35.20,209.0,-0.45,0.70,0.00,X40,NTT,FOOTWEAR
29,EASEMYTRIP,18.12,26.4,-9.80,M-SC,24.17,86846.0,-89969.0,170765.0,-2.20,-50.88,196.63,45.70,197.0,-0.53,0.67,1.02,XY24,NTT,TRAVEL
59,QUESS,296.83,986.0,-40.50,M-SC,42.02,59174.0,-5832.0,156758.0,-1.89,-8.97,264.91,232.18,198.0,-0.04,0.45,1.12,XY24,NTT,MISC
30,GREENPANEL,375.16,537.0,196.51,M-SC,5.74,132248.0,-50830.0,129801.0,-1.70,-27.76,98.15,43.14,230.0,-0.39,1.01,24.40,XY24,NTT,MISC
63,REPCOHOME,515.07,880.0,-59.02,H-SC,4.08,150484.0,-54514.0,199752.0,-1.47,-26.59,132.74,70.85,139.0,-0.27,1.15,21.48,XY24,NTT,FINANCE


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,93.17,M-SC,10.53,83224.0,-17553.0,17643.0,0.36,-17.42,21.20,0.09,243.0,-0.99,0.64,9.34,OX40N,NTT,DURABLES
84,UJJIVANSFB,52.77,60.0,85.53,H-SC,20.57,116883.0,-25596.0,45117.0,0.28,-17.96,38.60,13.70,163.0,-0.57,0.90,37.95,OX40N,NTT,BANKS
49,KANSAINER,299.63,340.0,-69.07,H-SC,2.52,213471.0,-56196.0,92518.0,-0.49,-20.84,43.34,13.47,142.0,-0.61,1.63,7.97,XY24,NTT,PAINTS
20,CERA,7989.07,9475.0,-20.53,H-SC,3.09,113499.0,-30304.0,57056.0,-0.79,-21.07,50.27,18.60,148.0,-0.53,0.87,25.25,OX40N,NTT,CERAMICS
43,INDIGOPNTS,1407.73,1408.0,109.26,M-SC,3.11,137305.0,-37254.0,37292.0,0.75,-21.34,27.16,0.02,221.0,-1.00,1.05,19.06,OX40N,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-28.36,H-LC,15.84,236574.0,-59360.0,153276.0,-0.08,-20.06,64.79,31.73,2.0,-0.39,1.81,1.00,X40,BTT,IT
45,INFY,1461.46,2275.00,-22.11,H-LC,10.63,255075.0,-6526.0,152152.0,0.06,-2.49,59.65,55.67,4.0,-0.04,1.95,3.46,X40,BTT,IT
89,VBL,492.64,671.64,-8.30,H-LC,10.41,278523.0,13483.0,82833.0,0.67,5.09,29.74,36.33,5.0,0.16,2.13,19.06,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-41.62,H-LC,10.16,198120.0,-18399.0,111304.0,0.17,-8.50,56.18,42.91,7.0,-0.17,1.52,3.76,AR,NTT,ELECTRICAL
57,NESTLEIND,2268.60,2755.00,-17.26,H-LC,3.59,127401.0,-138025.0,194936.0,-0.20,-52.00,153.01,21.44,8.0,-0.71,0.98,2.81,XY25,NTT,FMCG


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,10.86,H-MC,13.94,158189.0,31989.0,14190.0,-1.2,25.35,8.97,36.59,111.0,2.25,1.21,41.04,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,10.86,H-MC,13.94,158189.0,31989.0,14190.0,-1.20,25.35,8.97,36.59,111.0,2.25,1.21,41.04,XY24,NTT,HEALTHCARE
50,KPIGREEN,497.21,731.35,9.79,H-SC,12.71,130990.0,5693.0,53313.0,0.96,4.54,40.70,47.09,141.0,0.11,1.00,63.94,MH,ATH,POWER
15,BLUESTARCO,1646.70,2326.38,1.67,H-MC,7.16,175510.0,10840.0,57129.0,0.54,6.58,32.55,41.28,91.0,0.19,1.34,15.09,X40N,ATH,AC
19,CAMS,3643.00,5211.76,-5.83,H-SC,3.48,106985.0,4981.0,38943.0,1.38,4.88,36.40,43.06,123.0,0.13,0.82,22.84,X40N,ATH,MISC
35,HEROMOTOCO,4311.81,5949.07,-9.65,H-MC,7.43,167685.0,16772.0,40529.0,3.14,11.11,24.17,37.97,92.0,0.41,1.28,39.23,AR,ATH,AUTO


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-9.65,H-MC,7.43,167685.0,16772.0,40529.0,3.14,11.11,24.17,37.97,92.0,0.41,1.28,39.23,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-51.09,H-SC,12.20,136374.0,13038.0,120718.0,2.05,10.57,88.52,108.45,119.0,0.11,1.04,35.29,AR,ATH,MISC
50,KPIGREEN,497.21,731.35,9.79,H-SC,12.71,130990.0,5693.0,53313.0,0.96,4.54,40.70,47.09,141.0,0.11,1.00,63.94,MH,ATH,POWER
56,NATIONALUM,189.63,247.44,-49.46,H-MC,2.03,101495.0,991.0,29647.0,2.67,0.99,29.21,30.49,80.0,0.03,0.78,33.71,MH,ATH,METALS
91,WHIRLPOOL,1219.98,2270.00,-44.45,M-SC,4.00,93862.0,2364.0,76385.0,0.68,2.58,81.38,86.07,219.0,0.03,0.72,35.78,XR,NTT,DURABLES


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,21.85,72192.0,-41357.0,81411.0,5.17,-36.42,112.77,35.27,268.0,-0.51,0.55,76.69,XR,NTT,HOTELS
6,ASIANTILES,75.41,137.00,6838.89,L-SC,6.91,76108.0,-17702.0,94321.0,2.55,-18.87,123.93,81.67,270.0,-0.19,0.58,47.39,XR,NTT,CERAMICS
3,ALKYLAMINE,2347.98,4546.37,-1.65,H-SC,16.33,91461.0,-9502.0,104037.0,2.59,-9.41,113.75,93.63,149.0,-0.09,0.70,36.23,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-19.15,H-SC,14.96,91995.0,-5985.0,27856.0,0.54,-6.11,30.28,22.32,152.0,-0.21,0.70,33.42,XR,ATH,FINANCE
36,HINDALCO,651.95,756.01,-18.25,H-LC,6.04,112288.0,7976.0,8669.0,5.21,7.65,7.72,15.96,11.0,0.92,0.86,25.84,X5K,ATH,METALS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-9.65,H-MC,7.43,167685.0,16772.0,40529.0,3.14,11.11,24.17,37.97,92.0,0.41,1.28,39.23,AR,ATH,AUTO
51,LALPATHLAB,2838.53,3545.00,-25.14,H-SC,9.63,223584.0,30564.0,17484.0,0.98,15.83,7.82,24.89,134.0,1.75,1.71,42.88,X40N,NTT,HEALTHCARE
70,SHALBY,261.39,327.00,957.42,M-SC,7.11,143867.0,-37276.0,82738.0,0.78,-20.58,57.51,25.10,232.0,-0.45,1.10,15.29,XY24,NTT,HEALTHCARE
89,VBL,492.64,671.64,-8.30,H-LC,10.41,278523.0,13483.0,82833.0,0.67,5.09,29.74,36.33,5.0,0.16,2.13,19.06,X40N,ATH,FMCG
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,21.85,72192.0,-41357.0,81411.0,5.17,-36.42,112.77,35.27,268.0,-0.51,0.55,76.69,XR,NTT,HOTELS


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.43
1,25,41.11
2,50,70.92


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    39.46
LC    34.26
MC    26.29
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.94
X40N     15.49
X40      13.32
XY25     10.66
XR        8.99
AR        8.45
OX40N     7.55
X5K       2.29
MH        1.78
X200      1.40
SR        1.14
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    25.89
H-SC    24.57
H-MC    23.32
M-SC    13.40
M-LC     7.27
M-MC     2.63
L-SC     1.49
L-LC     1.10
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.22,-11.11,46.83
IT,11.92,-19.96,79.87
FINANCE,8.32,-19.94,64.31
BANKS,7.66,-14.13,62.59
MISC,5.87,-19.62,88.19
PAINTS,5.64,-18.43,41.77
HEALTHCARE,5.32,5.51,18.35
ELECTRICAL,4.70,-14.64,64.28
AUTO,4.51,-14.65,64.53


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2856523.0,23
XR,1118529.0,13
AR,1105088.0,10
X40,833784.0,10
XY25,657693.0,8
X40N,584570.0,12
OX40N,474159.0,10
SR,194825.0,2
X5K,139178.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2808004.0,28
M-SC,1496062.0,17
H-LC,1359591.0,18
H-MC,1331638.0,18
M-LC,427222.0,5
M-MC,334563.0,2
L-SC,265436.0,3
L-MC,58612.0,1
L-LC,43745.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      936148.0      6
M-SC       XY24      873216.0      7
H-SC       AR        672191.0      5
           XR        529870.0      6
H-LC       X40       521275.0      5
H-MC       XY24      497382.0      5
M-MC       XY24      334563.0      2
H-LC       XY25      259747.0      3
           AR        231999.0      2
H-MC       X40       230423.0      4
           XY25      221907.0      2
H-LC       X40N      220501.0      5
H-SC       X40N      218824.0      4
           OX40N     202833.0      4
           SR        194825.0      2
L-SC       XR        175732.0      2
M-LC       XY24      175378.0      2
M-SC       AR        160369.0      2
H-MC       X40N      145245.0      3
M-SC       OX40N     132473.0      4
M-LC       X5K       130509.0      1
M-SC       XR        125911.0      2
           XY25      122007.0      1
H-MC       XR        117356.0      1
M-LC       XR        111048.0      1
L-SC       OX40N      89704.0      1
M-SC       X40        82086.0      1
H-LC       X200       77564.0      1
L-MC       XR         58612.0      1
H-SC       MH         53313.0      1
H-MC       OX40N      49149.0      1
L-LC       XY25       43745.0      1
H-MC       AR         40529.0      1
H-LC       XY24       39836.0      1
H-MC       MH         29647.0      1
M-LC       XY25       10287.0      1
H-LC       X5K         8669.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
